In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('balanced_review.csv')

In [3]:
df.columns.tolist()

['overall', 'reviewText', 'summary']

In [4]:
df['overall'].value_counts()

3    264000
5    132000
4    132000
2    132000
1    132000
Name: overall, dtype: int64

In [5]:
df.isnull().any(axis = 0)

overall       False
reviewText     True
summary        True
dtype: bool

In [6]:
df.isnull().any(axis = 1)

0         False
1         False
2         False
3         False
4         False
          ...  
791995    False
791996    False
791997    False
791998    False
791999    False
Length: 792000, dtype: bool

In [7]:
df[df.isnull().any(axis = 1)]

,overall,reviewText,summary
404,5,Our family has loved and worn the Merrell Jun...,NaN
828,5,NaN,Five Stars
1452,5,Cushy comfy!!!,NaN
2449,5,NaN,Five Stars
2559,5,NaN,Five Stars
...,...,...,...
790138,1,NaN,One Star
790481,1,NaN,One Star
790676,1,No good,NaN
791218,1,NaN,In two Months they got ripped and I do not wa...


In [8]:
df.dropna(inplace = True)

In [9]:
df['overall'] != 3

0         True
1         True
2         True
3         True
4         True
          ... 
791995    True
791996    True
791997    True
791998    True
791999    True
Name: overall, Length: 791193, dtype: bool

In [10]:
df = df[df['overall'] != 3]

In [11]:
df.shape

(527353, 3)

In [12]:
df['overall'].value_counts()

2    131930
1    131863
4    131855
5    131705
Name: overall, dtype: int64

In [13]:
import numpy as np

In [14]:
df['Positivity'] = np.where(df['overall'] > 3, 1, 0 )

In [15]:
df['Positivity'].value_counts()

0    263793
1    263560
Name: Positivity, dtype: int64

#features - reviewText col
#labels - Positivity

In [16]:
df['reviewText'].head()

0    I wear a size 7 or 7.5 in shoes.  I ordered th...
1    The pants are well  made with quality material...
2    quality suspenders for sure but even better th...
3    More comfortable than a baseball cap.  This ca...
4    Great pantyhose.  Perfect fit.  They make your...
Name: reviewText, dtype: object

#Bag of Words model

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
df['reviewText'].head()

0    I wear a size 7 or 7.5 in shoes.  I ordered th...
1    The pants are well  made with quality material...
2    quality suspenders for sure but even better th...
3    More comfortable than a baseball cap.  This ca...
4    Great pantyhose.  Perfect fit.  They make your...
Name: reviewText, dtype: object

In [19]:
features_train, features_test, labels_train, labels_test = train_test_split(df['reviewText'], df['Positivity'], random_state = 42 ) 

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
vect = CountVectorizer().fit(features_train)

len(vect.get_feature_names())

vect.get_feature_names()[15000:15010]

In [22]:
features_train_vectorized = vect.transform(features_train)

#prepare the model
#KNN, SVM, Naive Bayes, Logistic Regression, Decision Tree, Random Forest, Xgboost
#version 01

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
model = LogisticRegression()

In [25]:
model.fit(features_train_vectorized, labels_train)

E:\New folder\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
prediction = model.predict(vect.transform(features_test))

In [27]:
from sklearn.metrics import roc_auc_score
roc_auc_score(labels_test, prediction)

0.8964493863904383

#TF-IDF - term frequency inverse document frequency
#version 02

In [28]:
features_train, features_test, labels_train, labels_test = train_test_split(df['reviewText'], df['Positivity'], random_state = 42 ) 

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
model = LogisticRegression()

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
vect = TfidfVectorizer(min_df = 5).fit(features_train)

In [33]:
features_train_vectorized = vect.transform(features_train)

In [34]:
model.fit(features_train_vectorized, labels_train)

E:\New folder\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [35]:
predictions = model.predict(vect.transform(features_test))

In [36]:
from sklearn.metrics import roc_auc_score

In [37]:
roc_auc_score(labels_test, predictions)

0.900582649326867

In [38]:
import pickle

In [39]:
pkl_filename = "pickle_model.pkl"

In [40]:
with open(pkl_filename, 'wb') as file:
    pickle.dump(model,file)